<a href="https://colab.research.google.com/github/rexian/ML/blob/main/langchain/nvidia/langchain_chains.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
%%capture
!pip install langchain-core
!pip install langchain_community langchain_huggingface
!pip install langchain-nvidia-ai-endpoints
!pip langchain-openai tiktoken faiss-cpu

In [2]:
import getpass
import os
from langchain_nvidia_ai_endpoints import ChatNVIDIA

if not os.environ.get("NVIDIA_API_KEY"):
  os.environ["NVIDIA_API_KEY"] = getpass.getpass("Enter API key for NVIDIA AI Endpoints: ")
llm = ChatNVIDIA(model="meta/llama3-70b-instruct")

Enter API key for NVIDIA AI Endpoints: ··········


# Chains in LangChain?

A chain is an end-to-end wrapper around multiple individual components which are executed in a defined order.

- Chains link multiple processes in a set sequence to create complex applications.
- They're useful for:
  - Dividing complicated tasks into simpler steps.
  - Maintaining context and memory across different steps.
  - Adding custom processing or checks between steps.
  - Simplifying the debugging of multi-step operations.

**Basic Chain Types:**

- **LLMChain**: Combines several language model calls.
- **RouterChain**: Directs tasks to different chains based on set conditions.
- **SimpleSequentialChain**: Executes chains one after another.
- **TransformChain**: Alters data between chain steps.

Chains integrate various components into a cohesive flow, enhancing the capabilities and flexibility of language model applications.


In [3]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate(
    input_variables=["topic"],
    template = "Write a comedic, parody rap about the following topic: {topic}"
)

# using LCEL
chain = prompt | llm

chain.invoke({"topic": "wasabi flavoured saki"})

AIMessage(content='(Verse 1)\nYo, listen up, I got a tale to tell\n\'Bout a drink that\'s hot, and I\'m not just swell\nIt\'s saki, but not just any kind\nGot my taste buds doin\' the wasabi grind\n\nI\'m talkin\' \'bout a flavor so bold\nMakes your nose hairs curl, your eyes get old\nIt\'s like a dragon\'s breath in a cup\nWasabi saki, it\'s like my taste buds got robbed\n\n(Chorus)\nWasabi saki, it\'s a crazy ride\nGot my mouth on fire, gotta take it in stride\nI\'m sweatin\' like a pig, got my nose on fleek\nWasabi saki, it\'s the bomb, but also kinda meh\n\n(Verse 2)\nI thought I was a pro, a saki connoisseur\nBut this wasabi flavor got me feelin\' like a rook\nI added it to my sushi, thought I was cool\nNow my mouth\'s on fire, like a fool\n\nI\'m coughin\' and hackin\', got my eyes on tears\nMy sinuses are wavin\' the white flag for years\nI\'m searchin\' for a glass of milk, gotta calm this down\nBut all I got is more wasabi, spinning me around\n\n(Bridge)\nI\'m a warrior, I can

In [5]:
from langchain.schema import StrOutputParser

chain = prompt | llm | StrOutputParser()

print(chain.invoke({"topic": "wasabi flavoured saki"}))

(Verse 1)
Yo, listen up, I got a tale to tell
'Bout a drink that's hot, and I'm not just swell
It's saki, but not just any kind
Got my taste buds doin' the wasabi grind

I'm talkin' 'bout a flavor so bold
Makes your nose hairs curl, your eyes get old
It's like a dragon's breath in a cup
Wasabi saki, it's like my taste buds got robbed

(Chorus)
Wasabi saki, it's a crazy ride
Got my mouth on fire, gotta take it in stride
I'm sweatin' like a pig, got my nose on fleek
Wasabi saki, it's the bomb, but also kinda meh

(Verse 2)
I thought I was a pro, a saki connoisseur
But this wasabi flavor got me feelin' like a rook
I added it to my sushi, thought I was cool
Now my mouth's on fire, like a fool

I'm coughin' and hackin', got my eyes on tears
My sinuses are wavin' the white flag for years
I'm searchin' for a glass of milk, gotta calm this down
But all I got is more wasabi, spinning me around

(Bridge)
I'm a warrior, I can take the heat
But this wasabi saki's got me on my knees, begging for re

In [8]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain_core.runnables import RunnableBranch

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are PaRappa the Rapper. You spit hot fire flows like lava."),
        ("human", "Write a comedic, parody rap about the following topic: {topic}"),
    ]
)

runnable = prompt | llm | StrOutputParser()
print(runnable.invoke({"topic": "wasabi flavoured saki"}))

Yo, what's good fam, it's your boy PaRappa the Rapper,
Here to spit some fire that's gonna make you scatter,
I'm talkin' 'bout a drink that's got the flow so tight,
Wasabi flavoured saki, day and night!

[Intro music plays, with a mix of traditional Japanese instruments and a hip-hop beat]

Verse 1:
I'm chillin' with my homies, sippin' on some saki,
When I stumbled upon this new flavor, oh boy, it's whack-y!
Wasabi flavoured, I'm like, "Hold up, what's the deal?"
My nose startin' to tingle, my mouth's on fiya, for real!

Chorus:
Wasabi flavoured saki, it's a crazy ride,
Got my taste buds doin' the karaoke slide,
It's hot, it's spicy, it's a flavorful crime,
But I'm sippin' on it slow, all the time!

Verse 2:
I add a dash of soy, try to calm the flames,
But this wasabi saki's got me wearin' nothin' but shame,
My stomach's doin' flips, my eyes are gettin' red,
I'm coughin' up a lung, but my rhymes are still ahead!

Chorus:
Wasabi flavoured saki, it's a crazy ride,
Got my taste buds doin'

In [7]:
for chunk in runnable.stream({"topic": "wasabi flavoured saki"}):
    print(chunk, end="", flush=True)

Yo, it's your boy PaRappa, the rapping sensation
Here to spit some heat about a crazy revelation
I'm talkin' 'bout a drink that's got me feelin' bold
Wasabi flavored saki, it's a story to be told

Verse 1:
I was at the sushi spot, feelin' quite serene
When I saw the menu, and my eyes got keen
Wasabi saki, I thought it was a joke, yo
But then I took a sip, and oh boy, it was a GO!

It was like a firehose, up in my grill
My sinuses were like, "Dude, what's the deal?!"
I coughed and I sputtered, my eyes got all red
I thought I was gonna die, in my sushi dread

Chorus:
Wasabi saki, it's a crazy ride
Got my taste buds doin' the cha cha slide
It's like a ninja kick, right to the face
Wasabi saki, I'm in a spicy place!

Verse 2:
I tried to play it cool, but I couldn't hide
My mouth was on fire, it was a nasty ride
I reached for the soy sauce, thinking it was the cure
But it just made it worse, I was a hot mess, for sho'

I started to sweat, my nose was runnin' wild
I looked like a Japanese ra

**Understanding Routing in LangChain**

**Routing Concept**
- **Purpose**: Adds structure to interactions with LLMs by guiding the flow based on previous step outcomes.

- **Essence**: Determines the next step in a chain dynamically, based on the result of the previous step.

**Methods for Routing**
1. **RunnableBranch Usage**:
   - Manages decision-making for the next step in a chain.

2. **Custom Factory Function**:
   - Crafts a runnable based on previous step input.

   - Crucial: The function should only create a runnable, not execute it.

**Example Application**
- **Two-Step Sequence**:

  - **Step 1**: Classifies a question into categories (literature, history, biology, philosophy, or other).

  - **Step 2**: Routes the classified question to a corresponding prompt chain tailored for the identified category.

**Application Goal**: Showcase both routing methods in a practical scenario to enhance interactions with language models.

In [9]:
literature_template = """You are a seasoned literature professor with decades of experience analyzing literary works. \
You have a knack for understanding complex narratives and characters and can provide insights into underlying themes and motifs.

Here is a passage or question about a literary work:
{input}
"""

literature_prompt = PromptTemplate.from_template(literature_template)


history_template = """You are a historian with extensive knowledge about world history. \
From ancient civilizations to modern times, you can provide context, insights, and explanations about historical events and figures.

Here is a question about history:
{input}
"""

history_prompt = PromptTemplate.from_template(history_template)

biology_template = """You are a biologist with a passion for understanding the intricacies of life. \
From cellular processes to ecosystem dynamics, you can elucidate biological phenomena with clarity.

Here is a question about biology:
{input}
"""

biology_prompt = PromptTemplate.from_template(biology_template)

philosophy_template = """You are a philosopher who has studied the great thinkers of the past and present. \
You enjoy discussing ethical dilemmas, existential questions, and the nature of reality.

Here is a philosophical query:
{input}
"""

philosophy_prompt = PromptTemplate.from_template(philosophy_template)

general_prompt = PromptTemplate.from_template(
    "You are a helpful assistant. Answer the question as accurately as you can.\n\n{input}"
)


**RunnableBranch Mechanics**

1. **Structure**: A list of (condition, runnable) pairs, plus a default runnable.

2. **Operation**
   - On invocation, sequentially evaluates each condition with the given input.

   - Executes the first runnable where its condition is True.
   
   - If no condition matches, the default runnable is executed.

**Purpose**: Ensures structured decision-making in chain execution, directing the flow based on specific conditions.

### Sequential Chains

**Basics of Sequential Chains:**

- Sequential chains are for when you need one language model's output to become another's input.
- They're like an assembly line, where each step's result is the starting point for the next.


In [14]:
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser

# Template for the initial rap
template_one = """
You are a Punjabi Jatt rapper, like AP Dhillon or Sidhu Moosewala.

Given a topic, it is your job to spit bars of hard-hitting, gritty, dope rap.

Topic: {topic}

Rap:
"""

# Template for the diss track
template_two = """
You are an extremely competitive Punjabi Rapper.

Given the rap from another rapper, it's your job to write a diss track which
tears apart the rap and shames the original rapper.

Rap:
{rap}
"""

# Create prompt templates from the defined templates
prompt_template_one = PromptTemplate.from_template(template_one)
prompt_template_two = PromptTemplate.from_template(template_two)

# Define the operation chain
chain = (
    {"rap": prompt_template_one | llm | StrOutputParser()}
    | prompt_template_two
    | llm
    | StrOutputParser()
)

In [15]:
for chunk in chain.stream({"topic": "Drinking Crown Royal and mobbin in my red Challenger"}):
    print(chunk, end="", flush=True)

(Diss Track)

Yo, listen up, I got a fact-check to spit
'Bout a wannabe rapper who thinks he's hit
Crown Royal, the whiskey of basic dudes
Thinkin' a red Challenger makes him look cool, what a fool

He's a Jatt on the decline, his flow is whack
Got his crew of nobodies, they're just a bunch of lack
They're sippin' on that Crown, thinkin' they're the best
But they're just a bunch of posers, tryin' to pass the test

His Challenger's red, but his rhymes are dead
Ridin' through the city, but his career's in his head
Crown Royal's got him feelin' like a joke
Thinkin' he's a star, but he's just a smoke

They're makin' their own way, but it's the wrong direction
They're the kings of nothin', just a bunch of rejection
Crown Royal's got 'em feelin' like they're lost
They're mobbin' through the city, but nobody knows the cost

So here's to the failure, and the lies that you tell
You're the one in control, of your mediocre bell
Crown Royal and your Challenger, that's all you need
To stay in your 

**Transformation in Component Chains**

**Role of Transformation**:
   - Adjusts inputs as they transition between different components.

**Example Scenario**:
   - **Task**: Handle a lengthy text.
   - **Transformation**: Filter to retain only the first three paragraphs.
   - **Next Steps**: Pass the shortened text through a series of steps for summarization.

**Goal**: Demonstrates how transformations can effectively tailor inputs for specific processing needs in a component chain.

In [16]:
!wget https://www.gutenberg.org/files/2680/2680-0.txt

with open("/content/2680-0.txt") as f:
    meditations = f.read()

--2025-03-28 01:12:24--  https://www.gutenberg.org/files/2680/2680-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 425351 (415K) [text/plain]
Saving to: ‘2680-0.txt’

2680-0.txt          100%[===================>] 415.38K  2.09MB/s    in 0.2s    

2025-03-28 01:12:25 (2.09 MB/s) - ‘2680-0.txt’ saved [425351/425351]



In [17]:
from langchain.schema import StrOutputParser
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""Summarize this text: {output_text} Summary:""")

runnable = (
    {"output_text": lambda text: "\n".join(text.split("\n")[921:1021])}
    | prompt
    | ChatNVIDIA(model="meta/llama3-70b-instruct")
    | StrOutputParser()
)
runnable.invoke(meditations)

"The text appears to be a philosophical and introspective passage, possibly from the Meditations of Marcus Aurelius. Here is a summary of the main ideas:\n\n* The author encourages the reader to live a life of gravity, simplicity, and justice, free from vanity, passion, and hypocrisy.\n* One should focus on their own life and not worry about the opinions of others, as true happiness comes from within.\n* The author warns against being distracted by external events and advises the reader to learn from their experiences and focus on their own soul.\n* They emphasize the importance of understanding the nature of the universe and one's place within it, and that one should strive to live in accordance with reason and nature.\n* The author compares sins committed through lust to those committed through anger, arguing that the former is more deserving of condemnation.\n* They encourage the reader to live each day as if it were their last, and to not fear death, as it is a natural part of life

In [19]:
rephrase = PromptTemplate.from_template("""Rephrase this text: {output_text}
In the style of a 90s gangster rapper passionately speaking to his homies.
Rephrased:""")

runnable = (
    {"output_text": lambda text: "\n".join(text.split("\n")[921:1021])}
    | rephrase
    | ChatNVIDIA(model="meta/llama3-70b-instruct")
    | StrOutputParser()
)

for chunk in runnable.stream(meditations):
    print(chunk, end="", flush=True)

Yo, listen up, my peeps! Let's get one thing straight, we gotta keep it real, keep it gangsta, keep it on a hundred, ya feel me? We gotta make sure we're livin' life like it's our last day on this earth, no specs, no fantasies, no hypotheticals, just straight up, raw, and uncut.

First off, we gotta take care of ourselves, our own souls, our own minds. Don't get caught up in what others think, don't let the haters bring you down, don't get distracted by the noise, stay focused, stay on track. You gotta be your own boss, your own guide, your own mentor.

And then, we gotta take a step back, and look at the bigger picture. What's the universe all about? What's our place in it? What's the nature of reality, and how do we fit in? We gotta know our role, our purpose, our destiny.

And let me tell you somethin', there's two kinds of sins, the ones we commit when we're angry, and the ones we commit when we're feelin' lustful. Now, the angry ones, they might be bad, but at least they're comin'